Transformers. Updated versions, 2, 3, and 4 with weight balancing, focal loss, and SMOTE



In [ ]:
import pandas as pd
from transformers import DistilBertTokenizerFast
import torch
from torch.utils.data import Dataset

data_path = './data/diabetes-dataset/diabetes_012_health_indicators_BRFSS2015.csv'
df = pd.read_csv(data_path)

print("Column Names:", df.columns)

X = df.drop(columns=['Diabetes_012']) 
y = df[['Diabetes_012']] 


print("Target Values Distribution:", y['Diabetes_012'].value_counts())


c:\Users\rockr\anaconda3\envs\yologpu1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Column Names: Index(['Diabetes_012', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker',
       'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')
Target Values Distribution: Diabetes_012
0.0    213703
2.0     35346
1.0      4631
Name: count, dtype: int64


checking columns names

In [2]:
print(X.columns)
print(y.columns)


Index(['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke',
       'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')
Index(['Diabetes_012'], dtype='object')


preprocess data

In [ ]:
import pandas as pd
from transformers import DistilBertTokenizerFast
import torch
from torch.utils.data import Dataset
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

data_path = './data/diabetes-dataset/diabetes_012_health_indicators_BRFSS2015.csv'
df = pd.read_csv(data_path)

print("Column Names:", df.columns)

X = df.drop(columns=['Diabetes_012'])
y = df[['Diabetes_012']]  

print("Target Values Distribution:", y['Diabetes_012'].value_counts())

tokenizer = DistilBertTokenizerFast.from_pretrained('./saved_model')

class DiabetesDataset(Dataset):
    def __init__(self, features_df, target_df, tokenizer, target_col):
        self.features_df = features_df
        self.labels = torch.tensor(target_df[target_col].values)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.features_df)

    def __getitem__(self, idx):
        features = self.features_df.iloc[idx]
        text = " ".join([str(val) for val in features])
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=128)
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']
        label = self.labels[idx].item()
        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.long)
        }

target_col = 'Diabetes_012'
diabetes_dataset = DiabetesDataset(X, y, tokenizer, target_col)

print(diabetes_dataset[0])

from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DistilBertForSequenceClassification.from_pretrained('./saved_model', num_labels=3).to(device)

classes = np.array([0, 1, 2])
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y[target_col])
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

training_args = TrainingArguments(
    output_dir='./results-v2',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    save_total_limit=2 
)

train_size = int(0.9 * len(X))
test_size = len(X) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(diabetes_dataset, [train_size, test_size])

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)




Column Names: Index(['Diabetes_012', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker',
       'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')
Target Values Distribution: Diabetes_012
0.0    213703
2.0     35346
1.0      4631
Name: count, dtype: int64
{'input_ids': tensor([ 101, 1015, 1012, 1014, 1015, 1012, 1014, 1015, 1012, 1014, 2871, 1012,
        1014, 1015, 1012, 1014, 1014, 1012, 1014, 1014, 1012, 1014, 1014, 1012,
        1014, 1014, 1012, 1014, 1015, 1012, 1014, 1014, 1012, 1014, 1015, 1012,
        1014, 1014, 1012, 1014, 1019, 1012, 1014, 2324, 1012, 1014, 2321, 1012,
        1014, 1015, 1012, 1014, 1014, 1012, 1014, 1023, 1012, 1014, 1018, 1012,
        1014, 1017, 1012, 1014,  102,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,

c:\Users\rockr\anaconda3\envs\yologpu1\Lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


train model

In [12]:
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of test samples: {len(test_dataset)}")
print(f"Using device: {device}")

trainer.train()

trainer.save_model('./saved_model-v2')
tokenizer.save_pretrained('./saved_model-v2')

Number of training samples: 228312
Number of test samples: 25368
Using device: cuda


  0%|          | 0/42810 [00:00<?, ?it/s]c:\Users\rockr\anaconda3\envs\yologpu1\Lib\site-packages\transformers\models\distilbert\modeling_distilbert.py:403: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
                                                     
  1%|          | 501/42810 [01:31<2:13:05,  5.30it/s]

{'loss': 0.9977, 'grad_norm': 1.7325440645217896, 'learning_rate': 1.976640971735576e-05, 'epoch': 0.04}


                                                      
  2%|▏         | 1001/42810 [03:03<2:11:35,  5.30it/s]

{'loss': 0.9435, 'grad_norm': 3.1131138801574707, 'learning_rate': 1.9532819434711517e-05, 'epoch': 0.07}


...

100%|██████████| 42810/42810 [2:19:28<00:00,  5.95it/s]























































































































































































































































































































































































































































































































































































































































































































































































































                                                       

                                             
100%|██████████| 42810/42810 [2:21:03<00:00,  5.95

{'eval_loss': 0.9457257986068726, 'eval_runtime': 93.6909, 'eval_samples_per_second': 270.763, 'eval_steps_per_second': 16.928, 'epoch': 3.0}


                                                       
100%|██████████| 42810/42810 [2:21:04<00:00,  5.06it/s]


{'train_runtime': 8464.3658, 'train_samples_per_second': 80.92, 'train_steps_per_second': 5.058, 'train_loss': 0.9615401191595586, 'epoch': 3.0}


('./saved_model-v2\\tokenizer_config.json',
 './saved_model-v2\\special_tokens_map.json',
 './saved_model-v2\\vocab.txt',
 './saved_model-v2\\added_tokens.json',
 './saved_model-v2\\tokenizer.json')

In [ ]:

trainer.save_model('./saved_model-v2')
tokenizer.save_pretrained('./saved_model-v2')

{'input_ids': tensor([ 101, 1015, 1015, 1015, 2871, 1015, 1014, 1014, 1014, 1014, 1015, 1014,
        1015, 1014, 1019, 2324, 2321, 1015, 1014, 1023, 1018, 1017,  102,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0

checking all good before training

In [5]:
# Verify Data Loaders
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of test samples: {len(test_dataset)}")
print(f"Using device: {device}")



Number of training samples: 228312
Number of test samples: 25368
Using device: cuda


evaluate model

In [14]:
from sklearn.metrics import classification_report, confusion_matrix

predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)
labels = predictions.label_ids

report = classification_report(labels, preds, target_names=['No Diabetes', 'Pre-Diabetic', 'Diabetic'])
matrix = confusion_matrix(labels, preds)

print(report)
print(matrix)


100%|██████████| 1586/1586 [01:30<00:00, 17.59it/s]

              precision    recall  f1-score   support

 No Diabetes       0.94      0.76      0.84     21375
Pre-Diabetic       0.00      0.00      0.00       448
    Diabetic       0.33      0.75      0.46      3545

    accuracy                           0.74     25368
   macro avg       0.42      0.50      0.43     25368
weighted avg       0.83      0.74      0.77     25368

[[16237     0  5138]
 [  203     0   245]
 [  903     0  2642]]



c:\Users\rockr\anaconda3\envs\yologpu1\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rockr\anaconda3\envs\yologpu1\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rockr\anaconda3\envs\yologpu1\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result

train dataset report

In [18]:
train_predictions = trainer.predict(train_dataset)
train_preds = train_predictions.predictions.argmax(-1)
train_labels = train_predictions.label_ids

train_report = classification_report(train_labels, train_preds, target_names=['No Diabetes', 'Pre-Diabetic', 'Diabetic'])
train_matrix = confusion_matrix(train_labels, train_preds)

print(train_report)
print(train_matrix)


100%|██████████| 14270/14270 [14:11<00:00, 16.76it/s]
c:\Users\rockr\anaconda3\envs\yologpu1\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rockr\anaconda3\envs\yologpu1\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

 No Diabetes       0.87      0.97      0.92    192328
Pre-Diabetic       0.00      0.00      0.00      4159
    Diabetic       0.55      0.23      0.32     31825

    accuracy                           0.85    228312
   macro avg       0.47      0.40      0.41    228312
weighted avg       0.81      0.85      0.82    228312

[[186932      0   5396]
 [  3663      0    496]
 [ 24644      0   7181]]


c:\Users\rockr\anaconda3\envs\yologpu1\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

report = classification_report(labels, preds, target_names=['No Diabetes', 'Pre-Diabetic', 'Diabetic'], zero_division=0)
matrix = confusion_matrix(labels, preds)

micro_precision, micro_recall, micro_f1, _ = precision_recall_fscore_support(labels, preds, average='micro')

matrix_percentage = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis] * 100

print("Classification Report:\n", report)
print("Confusion Matrix (Counts):\n", matrix)
print("Confusion Matrix (Percentages):\n", matrix_percentage)
print(f"Micro-Average Precision: {micro_precision:.2f}")
print(f"Micro-Average Recall: {micro_recall:.2f}")
print(f"Micro-Average F1-Score: {micro_f1:.2f}")

#  matrix as percentages
categories = ['No Diabetes', 'Pre-Diabetic', 'Diabetic']
print("\nConfusion Matrix as Percentages:")
for i, category in enumerate(categories):
    print(f"\nActual: {category}")
    for j, predicted_category in enumerate(categories):
        print(f"Predicted as {predicted_category}: {matrix_percentage[i][j]:.2f}%")


Classification Report:
               precision    recall  f1-score   support

 No Diabetes       0.94      0.76      0.84     21375
Pre-Diabetic       0.00      0.00      0.00       448
    Diabetic       0.33      0.75      0.46      3545

    accuracy                           0.74     25368
   macro avg       0.42      0.50      0.43     25368
weighted avg       0.83      0.74      0.77     25368

Confusion Matrix (Counts):
 [[16237     0  5138]
 [  203     0   245]
 [  903     0  2642]]
Confusion Matrix (Percentages):
 [[75.9625731   0.         24.0374269 ]
 [45.3125      0.         54.6875    ]
 [25.47249647  0.         74.52750353]]
Micro-Average Precision: 0.74
Micro-Average Recall: 0.74
Micro-Average F1-Score: 0.74

Confusion Matrix as Percentages:

Actual: No Diabetes
Predicted as No Diabetes: 75.96%
Predicted as Pre-Diabetic: 0.00%
Predicted as Diabetic: 24.04%

Actual: Pre-Diabetic
Predicted as No Diabetes: 45.31%
Predicted as Pre-Diabetic: 0.00%
Predicted as Diabetic: 54.6

new evaluation, version 2

In [ ]:
import pandas as pd
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
import torch
from torch.utils.data import Dataset
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
import numpy as np

tokenizer = DistilBertTokenizerFast.from_pretrained('./saved_model-v2')
model = DistilBertForSequenceClassification.from_pretrained('./saved_model-v2').to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

data_path = './data/diabetes-dataset/diabetes_012_health_indicators_BRFSS2015.csv'
df = pd.read_csv(data_path)

X = df.drop(columns=['Diabetes_012'])
y = df[['Diabetes_012']]

class DiabetesDataset(Dataset):
    def __init__(self, features_df, target_df, tokenizer, target_col):
        self.features_df = features_df
        self.labels = torch.tensor(target_df[target_col].values)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.features_df)

    def __getitem__(self, idx):
        features = self.features_df.iloc[idx]
        text = " ".join([str(val) for val in features])
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=128)
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']
        label = self.labels[idx].item()
        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.long)
        }

target_col = 'Diabetes_012'
diabetes_dataset = DiabetesDataset(X, y, tokenizer, target_col)

train_size = int(0.9 * len(X))
test_size = len(X) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(diabetes_dataset, [train_size, test_size])

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)
        labels = batch['labels'].to(model.device)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

# Generate classification report and confusion matrix
report = classification_report(all_labels, all_preds, target_names=['No Diabetes', 'Pre-Diabetic', 'Diabetic'], zero_division=0)
matrix = confusion_matrix(all_labels, all_preds)
micro_precision, micro_recall, micro_f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='micro')
matrix_percentage = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis] * 100

print("Classification Report:\n", report)
print("Confusion Matrix (Counts):\n", matrix)
print("Confusion Matrix (Percentages):\n", matrix_percentage)
print(f"Micro-Average Precision: {micro_precision:.2f}")
print(f"Micro-Average Recall: {micro_recall:.2f}")
print(f"Micro-Average F1-Score: {micro_f1:.2f}")

# Confusion matrix as percentages
categories = ['No Diabetes', 'Pre-Diabetic', 'Diabetic']
print("\nConfusion Matrix as Percentages:")
for i, category in enumerate(categories):
    print(f"\nActual: {category}")
    for j, predicted_category in enumerate(categories):
        print(f"Predicted as {predicted_category}: {matrix_percentage[i][j]:.2f}%")


c:\Users\rockr\anaconda3\envs\yologpu1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\rockr\anaconda3\envs\yologpu1\Lib\site-packages\transformers\models\distilbert\modeling_distilbert.py:403: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Classification Report:
               precision    recall  f1-score   support

 No Diabetes       0.94      0.77      0.84     21423
Pre-Diabetic       0.00      0.00      0.00       451
    Diabetic       0.33      0.75      0.46      3494

    accuracy                           0.75     25368
   macro avg       0.42      0.51      0.43     25368
weighted avg       0.84      0.75      0.78     25368

Confusion Matrix (Counts):
 [[16410     0  5013]
 [  196     0   255]
 [  875     0  2619]]
Confusion Matrix (Percentages):
 [[76.59991598  0.         23.40008402]
 [43.45898004  0.         56.54101996]
 [25.04293074  0.         74.95706926]]
Micro-Average Precision: 0.75
Micro-Average Recall: 0.75
Micro-Average F1-Score: 0.75

Confusion Matrix as Percentages:

Actual: No Diabetes
Predicted as No Diabetes: 76.60%
Predicted as Pre-Diabetic: 0.00%
Predicted as Diabetic: 23.40%

Actual: Pre-Diabetic
Predicted as No Diabetes: 43.46%
Predicted as Pre-Diabetic: 0.00%
Predicted as Diabetic: 56.5

Using focal loss to solve class imbalance, and creating saved_model-v3

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import torch.nn as nn
import numpy as np

# Focal Loss definition
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        device = inputs.device
        BCE_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-BCE_loss)
        if self.alpha is not None:
            self.alpha = self.alpha.to(device)  # Ensure alpha is on the same device
            alpha_t = self.alpha.gather(0, targets)
            F_loss = alpha_t * (1 - pt) ** self.gamma * BCE_loss
        else:
            F_loss = (1 - pt) ** self.gamma * BCE_loss
        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()
        else:
            return F_loss

# Adjust the alpha parameter to give higher weights to underrepresented classes
# [No Diabetes, Pre-Diabetic, Diabetic]
alpha = torch.tensor([1, 5, 3], dtype=torch.float32)  # giving higher weight to "Pre-Diabetic" and "Diabetic"
focal_loss = FocalLoss(gamma=2.0, alpha=alpha)

# Load tokenizer and model
tokenizer = DistilBertTokenizerFast.from_pretrained('./saved_model-v2')
model = DistilBertForSequenceClassification.from_pretrained('./saved_model-v2', num_labels=3)

# Load and prepare dataset
data_path = './data/diabetes-dataset/diabetes_012_health_indicators_BRFSS2015.csv'
df = pd.read_csv(data_path)

# Partition data
X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:, 1:], df[['Diabetes_012']], test_size=0.2, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

class DiabetesDataset(Dataset):
    def __init__(self, features_df, target_df, tokenizer, target_col):
        self.features_df = features_df
        self.labels = torch.tensor(target_df[target_col].values, dtype=torch.long)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.features_df)

    def __getitem__(self, idx):
        features = self.features_df.iloc[idx]
        text = " ".join([str(val) for val in features])
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=128)
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']
        label = self.labels[idx].item()
        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.long)
        }

target_col = 'Diabetes_012'
train_dataset = DiabetesDataset(X_train, y_train, tokenizer, target_col)
val_dataset = DiabetesDataset(X_val, y_val, tokenizer, target_col)
test_dataset = DiabetesDataset(X_test, y_test, tokenizer, target_col)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results-v3',
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Custom loss function for the Trainer
def custom_loss(outputs, labels):
    return focal_loss(outputs.logits, labels)

# Trainer class
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        loss = custom_loss(outputs, labels)
        return (loss, outputs) if return_outputs else loss

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset  # use validation dataset for evaluation
)



c:\Users\rockr\anaconda3\envs\yologpu1\Lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
trainer.train()

trainer.save_model('./saved_model-v3')
tokenizer.save_pretrained('./saved_model-v3')


  0%|          | 0/30444 [02:10<?, ?it/s]
                                         
  0%|          | 0/30444 [02:46<?, ?it/s]            

{'loss': 0.4303, 'grad_norm': 0.9933412671089172, 'learning_rate': 4.917882012876101e-05, 'epoch': 0.05}


                                         
  0%|          | 0/30444 [04:22<?, ?it/s]             

{'loss': 0.4299, 'grad_norm': 0.9223166704177856, 'learning_rate': 4.8357640257522006e-05, 'epoch': 0.1}


                                         
  0%|          | 0/30444 [06:00<?, ?it/s]             

...

                                         
  0%|          | 0/30444 [21:52<?, ?it/s]             

{'loss': 0.4357, 'grad_norm': 0.911104679107666, 'learning_rate': 4.0145841545132044e-05, 'epoch': 0.59}


                                         
  0%|          | 0/30444 [23:37<?, ?it/s]             

{'loss': 0.4202, 'grad_norm': 1.2117100954055786, 'learning_rate': 3.9324661673893056e-05, 'epoch': 0.64}


KeyboardInterrupt: 

Training takes excessively long this way, following is a faster way to train it.

Threfore to make it run faster we will implement the following to lower the time.

Learning rate: set as before to 2e-5.

Gradient Accumulation: Accumulate gradients over 4 steps with gradient_accumulation_steps=4.

Mixed Precision Training: Enabled mixed precision training with fp16=True.

In [15]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import torch.nn as nn
import numpy as np

# Focal Loss definition
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        device = inputs.device
        BCE_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-BCE_loss)
        if self.alpha is not None:
            self.alpha = self.alpha.to(device)  # Ensure alpha is on the same device
            alpha_t = self.alpha.gather(0, targets)
            F_loss = alpha_t * (1 - pt) ** self.gamma * BCE_loss
        else:
            F_loss = (1 - pt) ** self.gamma * BCE_loss
        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()
        else:
            return F_loss

# Adjust the alpha parameter to give higher weights to underrepresented classes
# [No Diabetes, Pre-Diabetic, Diabetic]
alpha = torch.tensor([1, 5, 3], dtype=torch.float32)  # giving higher weight to "Pre-Diabetic" and "Diabetic"
focal_loss = FocalLoss(gamma=2.0, alpha=alpha)

# Load tokenizer and model
tokenizer = DistilBertTokenizerFast.from_pretrained('./saved_model-v2')
model = DistilBertForSequenceClassification.from_pretrained('./saved_model-v2', num_labels=3)

# Load and prepare dataset
data_path = './data/diabetes-dataset/diabetes_012_health_indicators_BRFSS2015.csv'
df = pd.read_csv(data_path)

# Partition data
X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:, 1:], df[['Diabetes_012']], test_size=0.2, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

# Define your dataset class
class DiabetesDataset(Dataset):
    def __init__(self, features_df, target_df, tokenizer, target_col):
        self.features_df = features_df
        self.labels = torch.tensor(target_df[target_col].values, dtype=torch.long)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.features_df)

    def __getitem__(self, idx):
        features = self.features_df.iloc[idx]
        text = " ".join([str(val) for val in features])
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=128)
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']
        label = self.labels[idx].item()
        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.long)
        }

target_col = 'Diabetes_012'
train_dataset = DiabetesDataset(X_train, y_train, tokenizer, target_col)
val_dataset = DiabetesDataset(X_val, y_val, tokenizer, target_col)
test_dataset = DiabetesDataset(X_test, y_test, tokenizer, target_col)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results-v3',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    gradient_accumulation_steps=4,  # Accumulate gradients over 4 steps
    fp16=True  # Enable mixed precision training
)

# Custom loss function for the Trainer
def custom_loss(outputs, labels):
    return focal_loss(outputs.logits, labels)

# Define Trainer class
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        loss = custom_loss(outputs, labels)
        return (loss, outputs) if return_outputs else loss

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset  # use validation dataset for evaluation
)

trainer.train()

trainer.save_model('./saved_model-v3')
tokenizer.save_pretrained('./saved_model-v3')


c:\Users\rockr\anaconda3\envs\yologpu1\Lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/30444 [23:54<?, ?it/s]
                                                  
  7%|▋         | 500/7611 [02:26<34:50,  3.40it/s]    

{'loss': 1.5466, 'grad_norm': 3.0694215297698975, 'learning_rate': 1.8688739981605574e-05, 'epoch': 0.2}


                                                    
 13%|█▎        | 1000/7611 [04:56<31:50,  3.46it/s]   

{'loss': 1.4579, 'grad_norm': 2.710876226425171, 'learning_rate': 1.737485218762318e-05, 'epoch': 0.39}


                                                     
 20%|█▉        | 1500/7611 [07:25<30:09,  3.38it/s]   

...

                                                     
 99%|█████████▊| 7500/7611 [1:08:51<02:17,  1.24s/it]   

{'loss': 1.5014, 'grad_norm': 3.7993664741516113, 'learning_rate': 3.021941926159506e-07, 'epoch': 2.96}


100%|██████████| 7611/7611 [1:11:09<00:00,  1.19s/it]




















































































































































































































































































































































































































































































































































































































































































































































































































































































































































































{'eval_loss': 0.36899298429489136, 'eval_runtime': 222.4804, 'eval_samples_per_second': 182.439, 'eval_steps_per_second': 11.403, 'epoch': 3.0}
{'train_runtime': 4493.3901, 'train_samples_per_second': 108.396, 'train_steps_per_second': 1.694, 'train_loss': 1.4982005675539771, 'epoch': 3.0}


('./saved_model-v3\\tokenizer_config.json',
 './saved_model-v3\\special_tokens_map.json',
 './saved_model-v3\\vocab.txt',
 './saved_model-v3\\added_tokens.json',
 './saved_model-v3\\tokenizer.json')

testing new model v3

In [16]:
from sklearn.metrics import classification_report, confusion_matrix

predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)
labels = predictions.label_ids

report = classification_report(labels, preds, target_names=['No Diabetes', 'Pre-Diabetic', 'Diabetic'])
matrix = confusion_matrix(labels, preds)

print(report)
print(matrix)


100%|██████████| 3171/3171 [39:27:30<00:00, 44.80s/it]        


              precision    recall  f1-score   support

 No Diabetes       0.92      0.85      0.88     42795
Pre-Diabetic       0.00      0.00      0.00       944
    Diabetic       0.38      0.63      0.48      6997

    accuracy                           0.80     50736
   macro avg       0.43      0.49      0.45     50736
weighted avg       0.83      0.80      0.81     50736

[[36198     0  6597]
 [  532     0   412]
 [ 2621     0  4376]]


c:\Users\rockr\anaconda3\envs\yologpu1\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rockr\anaconda3\envs\yologpu1\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rockr\anaconda3\envs\yologpu1\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

In [17]:
!pip install imbalanced-learn

   ---------------------------------------- 0.0/258.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/258.3 kB ? eta -:--:--
   ----------------------- ---------------- 153.6/258.3 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 258.3/258.3 kB 2.6 MB/s eta 0:00:00


Using SMOTE

In [18]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight
import torch.nn as nn
import numpy as np

# Focal Loss definition
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        device = inputs.device
        BCE_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-BCE_loss)
        if self.alpha is not None:
            self.alpha = self.alpha.to(device)  # Ensure alpha is on the same device
            alpha_t = self.alpha.gather(0, targets)
            F_loss = alpha_t * (1 - pt) ** self.gamma * BCE_loss
        else:
            F_loss = (1 - pt) ** self.gamma * BCE_loss
        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()
        else:
            return F_loss

# Adjust the alpha parameter to give higher weights to underrepresented classes
# [No Diabetes, Pre-Diabetic, Diabetic]
alpha = torch.tensor([1, 5, 3], dtype=torch.float32)  # giving higher weight to "Pre-Diabetic" and "Diabetic"
focal_loss = FocalLoss(gamma=2.0, alpha=alpha)

# Load tokenizer and model
tokenizer = DistilBertTokenizerFast.from_pretrained('./saved_model-v3')
model = DistilBertForSequenceClassification.from_pretrained('./saved_model-v3', num_labels=3)

# Load and prepare dataset
data_path = './data/diabetes-dataset/diabetes_012_health_indicators_BRFSS2015.csv'
df = pd.read_csv(data_path)

# Partition data
X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:, 1:], df[['Diabetes_012']], test_size=0.2, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Define your dataset class
class DiabetesDataset(Dataset):
    def __init__(self, features_df, target_df, tokenizer, target_col):
        self.features_df = features_df
        self.labels = torch.tensor(target_df[target_col].values, dtype=torch.long)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.features_df)

    def __getitem__(self, idx):
        features = self.features_df.iloc[idx]
        text = " ".join([str(val) for val in features])
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=128)
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']
        label = self.labels[idx].item()
        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.long)
        }

target_col = 'Diabetes_012'
train_dataset = DiabetesDataset(X_train_smote, y_train_smote, tokenizer, target_col)
val_dataset = DiabetesDataset(X_val, y_val, tokenizer, target_col)
test_dataset = DiabetesDataset(X_test, y_test, tokenizer, target_col)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results-v4',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    gradient_accumulation_steps=4,  # Accumulate gradients over 4 steps
    fp16=True  # Enable mixed precision training
)

# Custom loss function for the Trainer
def custom_loss(outputs, labels):
    return focal_loss(outputs.logits, labels)

# Define Trainer class
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        loss = custom_loss(outputs, labels)
        return (loss, outputs) if return_outputs else loss

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset  # use validation dataset for evaluation
)

trainer.train()

trainer.save_model('./saved_model-v4')
tokenizer.save_pretrained('./saved_model-v4')



c:\Users\rockr\anaconda3\envs\yologpu1\Lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  3%|▎         | 500/19224 [02:17<1:24:58,  3.67it/s]   

{'loss': 2.3902, 'grad_norm': 13.41244125366211, 'learning_rate': 1.9481897627965046e-05, 'epoch': 0.08}


  5%|▌         | 1000/19224 [04:36<1:24:23,  3.60it/s]   

{'loss': 2.0319, 'grad_norm': 14.018302917480469, 'learning_rate': 1.8961714523512277e-05, 'epoch': 0.16}


  8%|▊         | 1500/19224 [06:57<1:22:09,  3.60it/s]   

...

 99%|█████████▉| 19000/19224 [1:31:30<01:00,  3.73it/s]  

{'loss': 1.3727, 'grad_norm': 29.635896682739258, 'learning_rate': 2.444860590928007e-07, 'epoch': 2.96}


100%|██████████| 19224/19224 [1:32:33<00:00,  3.65it/s]























































































































































































































































































































































































































































































































                                             
                                                         
100%|██████████| 19224/19224 [1:33:31<00:00,  3.65it/s]

100%|██████████| 19224/19224 [1:33:31<00:00,  3.43it/s]  


{'eval_loss': 0.3678590953350067, 'eval_runtime': 55.7158, 'eval_samples_per_second': 728.501, 'eval_steps_per_second': 45.535, 'epoch': 3.0}
{'train_runtime': 5611.827, 'train_samples_per_second': 219.241, 'train_steps_per_second': 3.426, 'train_loss': 1.6693153921088426, 'epoch': 3.0}


('./saved_model-v4\\tokenizer_config.json',
 './saved_model-v4\\special_tokens_map.json',
 './saved_model-v4\\vocab.txt',
 './saved_model-v4\\added_tokens.json',
 './saved_model-v4\\tokenizer.json')

In [ ]:

from sklearn.metrics import classification_report, confusion_matrix

predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)
labels = predictions.label_ids

report = classification_report(labels, preds, target_names=['No Diabetes', 'Pre-Diabetic', 'Diabetic'])
matrix = confusion_matrix(labels, preds)

print(report)
print(matrix)


100%|██████████| 3171/3171 [01:10<00:00, 44.79it/s]

              precision    recall  f1-score   support

 No Diabetes       0.92      0.85      0.88     42795
Pre-Diabetic       0.00      0.00      0.00       944
    Diabetic       0.38      0.61      0.47      6997

    accuracy                           0.80     50736
   macro avg       0.43      0.49      0.45     50736
weighted avg       0.83      0.80      0.81     50736

[[36291     0  6504]
 [  552     0   392]
 [ 2702     0  4295]]



c:\Users\rockr\anaconda3\envs\yologpu1\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rockr\anaconda3\envs\yologpu1\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rockr\anaconda3\envs\yologpu1\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result